# The Battle of Neighborhood 2

## Introduction

### This project is to take a glimpse at different neighborhoods of Providence,RI and Hartford,CT. Both cities are capital cities of their states and are major cities between New York City and Boston in East Coast.

### Travelers from inside and outside of New England area who want to dicide where to visit and business owners may be interested in the project.


## Data 

### I organized two data sets of zipcode and neighborhoods with latitude and longtude for each city from multiple online sources

##  Methodology

### Basic skills from week 3 lab
### Majorly relied on Foursquare API to retrieve all venues of each neighborhoods, then group by each neighborhoods and to count how many venues before filter top 10 most common venue types of each neighborhoods

In [1]:
## Data where you describe the data that will be used to solve the problem and the source of the data.

In [2]:
# Import the required library
import numpy as np
import pandas as pd

In [3]:
# Load data of Providence,RI
df_pvd=pd.read_excel("pvd.xlsx")
df_pvd.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pvd.xlsx'

In [ ]:
# Add '0' before '2' of each zipcode
df_pvd['Zipcode']='0'+ df_pvd['Zipcode'].astype(str)

In [ ]:
df_pvd.head()

In [ ]:
df_pvd.shape

In [ ]:
# Load data of Hartford,CT
df_bdl=pd.read_excel("bdl.xlsx")
df_bdl.head()

In [ ]:
# Add '0' before '6' of each zipcode
df_bdl['Zipcode']='0'+ df_bdl['Zipcode'].astype(str)

In [ ]:
df_bdl.head()

In [ ]:
df_bdl.shape

In [ ]:
# Import mapping libraries
from geopy.geocoders import Nominatim
import folium

In [ ]:
# Importing libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

In [ ]:
# Working with Foursquare
CLIENT_ID = 'FD5IPVIXWPQA0FVEOHMZPM0MXPQIHKDQPO20JMZMCK1EA2LN' # your Foursquare ID
CLIENT_SECRET = 'SXF5QRXJ4T1UVTLZFOXXSTVBXA54FMAOE1PMI0CP5X2IKAP0' # your Foursquare Secret
VERSION = '20181224' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
pvd_venues=getNearbyVenues(df_pvd['Neighborhoods'], df_pvd['Latitude'], df_pvd['Longitude'])
bdl_venues=getNearbyVenues(df_bdl['Neighborhoods'], df_bdl['Latitude'], df_bdl['Longitude'])

In [ ]:
pvd_venues.head()

In [ ]:
bdl_venues.head()

In [ ]:
pvd_venues.shape

In [ ]:
bdl_venues.shape

In [ ]:
# Number of venues of each 
pvd_venues.groupby('Neighborhood').count()

In [ ]:
bdl_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
pvd_onehot = pd.get_dummies(pvd_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pvd_onehot['Neighborhood'] = pvd_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [pvd_onehot.columns[-1]] + list(pvd_onehot.columns[:-1])
pvd_onehot = pvd_onehot[fixed_columns]

pvd_onehot.head()

In [ ]:
pvd_grouped = pvd_onehot.groupby('Neighborhood').mean().reset_index()
pvd_grouped.head()

In [ ]:
# one hot encoding
bdl_onehot = pd.get_dummies(bdl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bdl_onehot['Neighborhood'] = bdl_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bdl_onehot.columns[-1]] + list(bdl_onehot.columns[:-1])
bdl_onehot = bdl_onehot[fixed_columns]

bdl_onehot.head()

In [ ]:
bdl_grouped = bdl_onehot.groupby('Neighborhood').mean().reset_index()
bdl_grouped.head()

In [ ]:
# Define a function for top venus for a neighborhood
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pvd_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
pvd_neighborhoods_venues_sorted['Neighborhood'] = pvd_grouped['Neighborhood']

for ind in np.arange(pvd_grouped.shape[0]):
    pvd_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pvd_grouped.iloc[ind, :], num_top_venues)

pvd_neighborhoods_venues_sorted.head()

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

bdl_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
bdl_neighborhoods_venues_sorted['Neighborhood'] = bdl_grouped['Neighborhood']

for ind in np.arange(bdl_grouped.shape[0]):
    bdl_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bdl_grouped.iloc[ind, :], num_top_venues)

bdl_neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

pvd_grouped_clustering = pvd_grouped.drop('Neighborhood', 1)

# run k-means clustering
pvd_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pvd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
pvd_kmeans.labels_[0:10] 

In [4]:
pvd_merged = df_pvd


pvd_merged['Cluster Labels'] = pvd_kmeans.labels_


pvd_merged = pvd_merged.join(pvd_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


pvd_merged.head()

NameError: name 'df_pvd' is not defined